In [1]:
# MLP for Pima Indians Dataset Serialize to JSON and HDF5

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.models import model_from_json
import numpy
import os
# fix random seed for reproducibility
numpy.random.seed(7)

# load pima indians dataset

import matplotlib.pyplot as plt
import pandas as pd
col_names = ['pregnant', 'glucose', 'bp', 'skin', 'insulin', 'bmi', 'pedigree', 'age', 'label']
df = pd.read_csv('pima-indians-diabetes.data.csv', names=col_names)
# # load pima indians dataset
# dataset = numpy.loadtxt("pima-indians-diabetes.data.csv", delimiter=",")

# selection of relevant features
label = df['label']
df.drop('label', axis=1, inplace=True)

X, Y = df, label
df.describe()

# normalized data
df_norm = (df - df.mean()) / (df.max() - df.min())
df_norm.describe()
# X = df[['glucose','bmi']]

Using TensorFlow backend.


In [14]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=0)

from sklearn.preprocessing import Imputer
# Cover n/a values with median values
med_imp = Imputer(missing_values=0, strategy='median', axis=0)
X_train.iloc[:, 1:6] = med_imp.fit_transform(X_train.iloc[:, 1:6])
X_test.iloc[:, 1:6] = med_imp.transform(X_test.iloc[:, 1:6])

/usr/local/lib/python2.7/dist-packages/pandas/core/indexing.py:537: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [15]:
# Standardize all explanatory variables
std_scl = StandardScaler()
X_train = std_scl.fit_transform(X_train)
X_test = std_scl.transform(X_test)

In [18]:
from sklearn.metrics import confusion_matrix
# Classify by SVM
svc = SVC()
svc.fit(X_train, y_train)
print('Train score: {:.3f}'.format(svc.score(X_train, y_train)))
print('Test score: {:.3f}'.format(svc.score(X_test, y_test)))
print('Confusion matrix:\n{}'.format(confusion_matrix(y_test, svc.predict(X_test))))

Train score: 0.764
Test score: 0.758
Confusion matrix:
[[141  16]
 [ 40  34]]


In [19]:
# Search hyper patameters, C and gamma
svc_param_grid = {
    'C': [0.001, 0.01, 0.1, 1, 10, 100],
    'gamma': [0.001, 0.01, 0.1, 1, 10, 100]
}

svc_grid_search = GridSearchCV(SVC(), svc_param_grid, cv=10)
svc_grid_search.fit(X_train, y_train)

print('Best parameters: {}'.format(svc_grid_search.best_params_))
print('Train score: {:.3f}'.format(svc_grid_search.score(X_train, y_train)))
print('Test score: {:.3f}'.format(svc_grid_search.score(X_test, y_test)))
print('Confusion matrix:\n{}'.format(confusion_matrix(y_test, svc_grid_search.predict(X_test))))

Best parameters: {'C': 1, 'gamma': 0.01}
Train score: 0.756
Test score: 0.762
Confusion matrix:
[[143  14]
 [ 41  33]]


In [11]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.grid_search import GridSearchCV
from sklearn.svm import SVC
pipe_svc = Pipeline([('scl', StandardScaler()),('clf', SVC(random_state=1))])
param_range = [0.0001, 0.001, 0.01, 0.1, 1.0, 10.0, 100.0, 1000.0]
param_grid = [{'clf__C': param_range,'clf__kernel': ['linear']},{'clf__C': param_range,'clf__gamma': param_range,'clf__kernel': ['rbf']}]
gs = GridSearchCV(estimator=pipe_svc,param_grid=param_grid,scoring='accuracy',cv=10,n_jobs=-1)
gs = gs.fit(X_train, y_train)

In [12]:
print(gs.best_score_)
print(gs.best_params_)

0.75791433892
{'clf__C': 0.01, 'clf__kernel': 'linear'}


In [13]:
clf = gs.best_estimator_
clf.fit(X_train, y_train)
print('Test accuracy: %.3f' % clf.score(X_test, y_test))

Test accuracy: 0.766
